In [1]:
import pandas as pd
import seaborn as sns

In [2]:
avengersEndgame = pd.read_csv('Avengers Endgame.csv')
forrestGump = pd.read_csv('Forrest Gump.csv')
johnWickChapterThree = pd.read_csv('John Wick Chapter 3.csv')
joker = pd.read_csv('Joker.csv')
morbius = pd.read_csv('Morbius.csv')
pulpFiction = pd.read_csv('Pulp Fiction.csv')
spiderManNoWayHome = pd.read_csv('SpiderMan No Way Home.csv')
theAvengers = pd.read_csv('The Avengers.csv')
theDarkKnight = pd.read_csv('The Dark Knight.csv')
thorRagnarok = pd.read_csv('Thor Ragnarok.csv')

In [3]:
df = pd.concat([avengersEndgame, forrestGump, johnWickChapterThree, joker, morbius, pulpFiction, spiderManNoWayHome, theAvengers, theDarkKnight, thorRagnarok], ignore_index=True)

In [4]:
#df = df.drop(columns = ['Date of Review', 'User', 'Usefulness Vote', 'Total Votes'])

In [5]:
df = df.rename(columns = {"User's Rating out of 10":"Rating"})

In [6]:
df.isnull().values.any()

True

In [7]:
df.dropna(inplace = True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47061 entries, 0 to 47062
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date of Review   47061 non-null  object
 1   User             47061 non-null  object
 2   Usefulness Vote  47061 non-null  int64 
 3   Total Votes      47061 non-null  int64 
 4   Rating           47061 non-null  object
 5   Review Title     47061 non-null  object
 6   Review           47061 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.9+ MB


In [9]:
df['Review'] = df['Review'].astype(str)

In [10]:
df['Review Title'] = df['Review Title'].astype(str)

In [11]:
df['Rating'] = df['Rating'].str.strip()

In [12]:
df = df[df.Rating != "Was this review helpful?  Sign in to vote."]

In [13]:
df['Rating'] = df['Rating'].astype(int)

In [14]:
df.isnull().values.any()

False

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45779 entries, 0 to 47062
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date of Review   45779 non-null  object
 1   User             45779 non-null  object
 2   Usefulness Vote  45779 non-null  int64 
 3   Total Votes      45779 non-null  int64 
 4   Rating           45779 non-null  int32 
 5   Review Title     45779 non-null  object
 6   Review           45779 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 2.6+ MB


In [16]:
df.reset_index(drop=True, inplace=True)

In [17]:
df.head()

,Date of Review,User,Usefulness Vote,Total Votes,Rating,Review Title,Review
0,4 May 2019,Paynebyname,1481,2771,4,"The writers got carried away, the directors ov...",I've just come from watching Endgame and I mus...
1,6 May 2019,jtindahouse,75,127,4,Time travel is such a lazy way to write stories,Only a month or so back I was talking to a fri...
2,13 May 2019,arclinecreative,100,170,3,Overrated and full of filler,Continuity? 350M budget and no one looking ove...
3,3 August 2019,RetroRick,99,169,3,"downbeat, overblown & so so long. Cuts all you...",Maybe I would have enjoyed this more if I were...
4,13 December 2021,ACollegeStudent,27,41,8,Not as good as infinity war but a great movie,Rating: 8.6Not as good as Infinity war pacing-...


In [18]:
len(df)

45779

In [19]:
def dropNonUsefulRows (df):
    for row in range(len(df.index)):
        ratio = None
        if df['Total Votes'][row] == 0:
            pass            
        elif df['Usefulness Vote'][row]/df['Total Votes'][row] > 0.5:
            df = df.drop(row)
            
    return df

In [20]:
df = dropNonUsefulRows (df)

In [21]:
len(df)

31805

In [22]:
df.describe()

,Usefulness Vote,Total Votes,Rating
count,31805.000000,31805.000000,31805.000000
mean,4.873542,13.404748,8.088602
std,22.039603,50.872017,2.663425
min,0.000000,0.000000,1.000000
25%,0.000000,2.000000,7.000000
50%,1.000000,4.000000,9.000000
75%,3.000000,9.000000,10.000000
max,1104.000000,2318.000000,10.000000


In [23]:
def fixPunctuation (allTexts):
    allTexts = allTexts.replace(".", " . ")
    allTexts = allTexts.replace(",", " , ")
    allTexts = allTexts.replace("?", " ? ")
    allTexts = allTexts.replace("!", " ! ")
    allTexts = allTexts.replace(":", " : ")
    allTexts = allTexts.replace(";", " ; ")
    allTexts = allTexts.replace("/", " / ")
    allTexts = allTexts.replace(")", " ) ")
    allTexts = allTexts.replace("(", " ( ")
    allTexts = allTexts.replace("'", " ' ")
    allTexts = allTexts.replace("'", " ' ")
    
    
    return allTexts

In [24]:
def makeColumns (df):
    allTexts = ""
    reviewTitle = ' '.join(df["Review Title"])
    review = ' '.join(df["Review"])
    allTexts = reviewTitle + " " + review
    
    allTexts = allTexts.lower()
    
    allTexts = fixPunctuation (allTexts)
        
    allTexts = allTexts.split()
    
    dic = {}
        
    for word in allTexts:
        if word in dic:
            x += 1
        else:
            x = 1
    
        dic[word] = x
        
    nonRelevantWords = [key for key, val in dic.items() if val <= 1]
    relevantWords = {key:[] for key, val in dic.items() if val > 1}
    
    relevantWords = pd.DataFrame.from_dict(relevantWords)
    
    df = pd.concat([df, relevantWords], ignore_index=True)
    
    return df, nonRelevantWords

In [25]:
df, nonRelevantWords = makeColumns (df)

In [26]:
def getValues (df, nonRelevantWords):
    end = len(df)
    
    for i in range (end):
        wordsForRow = df["Review Title"][i] + " " + df["Review"][i]
        
        wordsForRow = wordsForRow.lower()
    
        wordsForRow = fixPunctuation (wordsForRow)
        
        wordsForRow = wordsForRow.split()
        
        unrepeatedWords = list(set(wordsForRow))
        
        for word in unrepeatedWords:
            if word not in nonRelevantWords:
                df.loc[i, word] = wordsForRow.count(word)
            else:
                pass
    
    df = df.fillna(0)
    
    return df

In [27]:
df = getValues (df, nonRelevantWords)

KeyboardInterrupt: 

In [ ]:
df

In [ ]:
cor = df.corr()

In [ ]:
cor

In [ ]:
for column in cor:
    if not(abs(cor['Rating'][column]) >= 0.01):
        df = df.drop(columns = column)

In [ ]:
df